# 1b_Preprocessing

---

- Data Cleaning and Feature Engineering

In [21]:
import pandas as pd


## Import Data

In [22]:
df_train = pd.read_csv('../assets/data_wk/train.csv')
df_test = pd.read_csv('../assets/data_wk/test.csv')
df_spray = pd.read_csv('../assets/data_wk/spray.csv')
df_weather = pd.read_csv('../assets/data_wk/weather.csv')

In [23]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    10506 non-null  object 
 1   Address                 10506 non-null  object 
 2   Species                 10506 non-null  object 
 3   Block                   10506 non-null  int64  
 4   Street                  10506 non-null  object 
 5   Trap                    10506 non-null  object 
 6   AddressNumberAndStreet  10506 non-null  object 
 7   Latitude                10506 non-null  float64
 8   Longitude               10506 non-null  float64
 9   AddressAccuracy         10506 non-null  int64  
 10  NumMosquitos            10506 non-null  int64  
 11  WnvPresent              10506 non-null  int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 985.1+ KB


In [24]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116293 entries, 0 to 116292
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Id                      116293 non-null  int64  
 1   Date                    116293 non-null  object 
 2   Address                 116293 non-null  object 
 3   Species                 116293 non-null  object 
 4   Block                   116293 non-null  int64  
 5   Street                  116293 non-null  object 
 6   Trap                    116293 non-null  object 
 7   AddressNumberAndStreet  116293 non-null  object 
 8   Latitude                116293 non-null  float64
 9   Longitude               116293 non-null  float64
 10  AddressAccuracy         116293 non-null  int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 9.8+ MB


In [25]:
df_spray.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       14835 non-null  object 
 1   Time       14251 non-null  object 
 2   Latitude   14835 non-null  float64
 3   Longitude  14835 non-null  float64
dtypes: float64(2), object(2)
memory usage: 463.7+ KB


In [26]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Station      2944 non-null   int64  
 1   Date         2944 non-null   object 
 2   Tmax         2944 non-null   int64  
 3   Tmin         2944 non-null   int64  
 4   Tavg         2944 non-null   object 
 5   Depart       2944 non-null   object 
 6   DewPoint     2944 non-null   int64  
 7   WetBulb      2944 non-null   object 
 8   Heat         2944 non-null   object 
 9   Cool         2944 non-null   object 
 10  Sunrise      2944 non-null   object 
 11  Sunset       2944 non-null   object 
 12  CodeSum      2944 non-null   object 
 13  Depth        2944 non-null   object 
 14  Water1       2944 non-null   object 
 15  SnowFall     2944 non-null   object 
 16  PrecipTotal  2944 non-null   object 
 17  StnPressure  2944 non-null   object 
 18  SeaLevel     2944 non-null   object 
 19  Result

## Data Cleaning

---


### Change Date to DateTime datatype & Index

In [27]:
df_train['Date'] = pd.to_datetime(df_train['Date'])
df_test['Date'] = pd.to_datetime(df_test['Date'])
df_weather['Date'] = pd.to_datetime(df_weather['Date'])
df_spray['Date'] = pd.to_datetime(df_spray['Date'])

df_weather = df_weather.set_index('Date')
df_train = df_train.set_index('Date')
df_test = df_test.set_index('Date')
df_spray = df_spray.set_index('Date')


### Weather data - Drop Features and Convert Datatype to numeric

In [28]:
def float_convertor(string_value):
    try:
        return float(string_value)
    except:
        pass
    
def int_convertor(string_value):
    try:
        return int(string_value)
    except:
        try:
            string_value = string_value.replace(' ','')
            return int(string_value)
        except:
            pass

In [29]:
# reset index back to number of rows
df_weather = df_weather.reset_index()

# Drop rows with missing values indeicated as "M" or "-"
df_weather = df_weather.drop(df_weather[df_weather['StnPressure']=='M'].index, axis = 0)
df_weather = df_weather.drop(df_weather[df_weather['SeaLevel']=='M'].index, axis = 0)
df_weather = df_weather.drop(df_weather[df_weather['WetBulb']=='M'].index, axis = 0)
df_weather = df_weather.drop(df_weather[df_weather['Sunrise']=='-'].index, axis = 0)

# Convert string to integer or float
df_weather['Tavg'] = df_weather['Tavg'].map(int_convertor)
df_weather["StnPressure"]=df_weather['StnPressure'].map(float_convertor)
df_weather['SeaLevel']=df_weather['SeaLevel'].map(float_convertor)
df_weather['WetBulb'] = df_weather['WetBulb'].map(int_convertor)
df_weather['AvgSpeed'] = df_weather['AvgSpeed'].map(float_convertor)
df_weather['Heat'] = df_weather['Heat'].map(int_convertor)
df_weather['Cool'] = df_weather['Cool'].map(int_convertor)
df_weather['Sunrise']=df_weather['Sunset'].map(int_convertor)
df_weather['Sunset']=df_weather['Sunset'].map(int_convertor)

# Revert index back to date
df_weather = df_weather.set_index('Date')


### Convert spray data time feature to 24 hr format

In [30]:
df_spray['Time'] = pd.to_datetime(df_spray['Time'],format= '%I:%M:%S %p' ).dt.time
df_spray

,Time,Latitude,Longitude
Date,,,
2011-08-29,18:56:58,42.391623,-88.089163
2011-08-29,18:57:08,42.391348,-88.089163
2011-08-29,18:57:18,42.391022,-88.089157
2011-08-29,18:57:28,42.390637,-88.089158
2011-08-29,18:57:38,42.390410,-88.088858
...,...,...,...
2013-09-05,20:34:11,42.006587,-87.812355
2013-09-05,20:35:01,42.006192,-87.816015
2013-09-05,20:35:21,42.006022,-87.817392


## Feature Engineering

---

### Merge train/test and weather dataframe

In [31]:
df_main_train = pd.merge(df_train,df_weather,how='inner',left_index=True,right_index=True)


In [32]:
df_main_test = pd.merge(df_test,df_weather,how='inner',left_index=True,right_index=True)


### Extract month and day from date

In [33]:
df_main_test['Month'] = pd.to_datetime(df_main_test.index).month
df_main_test['Day'] = pd.to_datetime(df_main_test.index).day

df_main_train['Month'] = pd.to_datetime(df_main_train.index).month
df_main_train['Day'] = pd.to_datetime(df_main_train.index).day

### Add lag features

---

The lag features and durations are added based on the reseach done by another study. (Source: https://link.springer.com/article/10.1007/s00484-020-02059-9)

In [34]:
def add_lag_feature(df):
    
    for i in [7,14,21,28,35]:
        df[f'Tavg_Lag{i}'] = df['Tavg'].shift(i)
        df[f'DewPoint_Lag{i}'] = df['DewPoint'].shift(i)
        df[f'WetBulb_Lag{i}'] = df['WetBulb'].shift(i)
        df[f'PrecipTotal_Lag{i}'] = df['PrecipTotal'].shift(i)
        df[f'SeaLevel_Lag{i}'] = df['SeaLevel'].shift(i)
    return df

In [35]:
df_main_train = add_lag_feature(df_main_train)
df_main_test = add_lag_feature(df_main_test)

### Drop Features


In [36]:
df_main_train = df_main_train.drop(columns=['Address','Street','AddressNumberAndStreet','NumMosquitos','Trap'])
df_main_train = df_main_train.drop(columns=['CodeSum','Station','Heat','Cool','Sunrise','Sunset','SnowFall','Water1','Depth','Depart'])

df_main_test = df_main_test.drop(columns=['Address','Street','AddressNumberAndStreet','Trap'])
df_main_test = df_main_test.drop(columns=['CodeSum','Station','Heat','Cool','Sunrise','Sunset','SnowFall','Water1','Depth','Depart'])


In [37]:
df_main_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10413 entries, 2007-05-29 to 2013-09-26
Data columns (total 44 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Species            10413 non-null  object 
 1   Block              10413 non-null  int64  
 2   Latitude           10413 non-null  float64
 3   Longitude          10413 non-null  float64
 4   AddressAccuracy    10413 non-null  int64  
 5   WnvPresent         10413 non-null  int64  
 6   Tmax               10413 non-null  int64  
 7   Tmin               10413 non-null  int64  
 8   Tavg               10413 non-null  int64  
 9   DewPoint           10413 non-null  int64  
 10  WetBulb            10413 non-null  int64  
 11  PrecipTotal        10413 non-null  object 
 12  StnPressure        10413 non-null  float64
 13  SeaLevel           10413 non-null  float64
 14  ResultSpeed        10413 non-null  float64
 15  ResultDir          10413 non-null  int64  
 16  AvgSp

## Save Data

In [38]:
df_main_train.to_csv('../assets/data_wk/train_data_engineered.csv',index=False)
df_main_test.to_csv('../assets/data_wk/test_data_engineered.csv',index=False)
df_weather.to_csv('../assets/data_wk/weather_data_cleaned.csv',index=True)
df_spray.to_csv('../assets/data_wk/spray_data_cleaned.csv',index=True)
df_train.to_csv('../assets/data_wk/train_data_cleaned.csv',index=True)